In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

# crea el contexto de spark, maneja el cluster y las tareas
sc = SparkContext()
# para trabajar con dataframes y datasets
spark = SparkSession.builder.getOrCreate()

In [18]:
#pasar un txt a rdd
textfile = sc.textFile("/content/datasets/ejemplo.txt")

#no se muestra hasta que no ocurra una accion
textfile

/content/datasets/ejemplo.txt MapPartitionsRDD[49] at textFile at NativeMethodAccessorImpl.java:0

In [16]:
#pase csv a rdd
#para este caso es mejor abrirlo con un metodo distinto

cvsfile = spark.read.option("header", True).csv("/content/datasets/ratings.csv")
cvsfile.take(5)

[Row(userId='1', movieId='1', rating='4.0', timestamp='964982703'),
 Row(userId='1', movieId='3', rating='4.0', timestamp='964981247'),
 Row(userId='1', movieId='6', rating='4.0', timestamp='964982224'),
 Row(userId='1', movieId='47', rating='5.0', timestamp='964983815'),
 Row(userId='1', movieId='50', rating='5.0', timestamp='964982931')]

In [22]:
#toma solo los elementos de la columna movieId

# Leer el archivo CSV
df = spark.read.csv("/content/datasets/ratings.csv", header=True, inferSchema=True)

# Nombre de la columna que deseas seleccionar
column_name = "movieId"

# Seleccionar la columna y convertirla a un RDD
column_movieId = df.select(column_name).rdd.map(lambda row: row[0])

# Mostrar los primeros elementos del RDD
print(column_movieId.take(10))

[1, 3, 6, 47, 50, 70, 101, 110, 151, 157]


# Acciones

In [6]:
#para mostrar
textfile.collect()

['primera linea ', 'segunda linea', 'tercera linea', 'cuarta linea']

In [7]:
#muetra el primer elemento
textfile.first()

'primera linea '

In [19]:
#imprime un numero determinado de lineas
textfile.take(3)

['primera linea ', 'segunda linea', 'tercera linea']

In [8]:
#cuenta los elementos
textfile.count()

4

# Transformaciones


In [21]:
#Filtrado por la palabra segunda
segunda = textfile.filter( lambda  linea : "segunda" in linea )
segunda.collect()

['segunda linea']

In [26]:
# map()

#suma 1
def suma(x):
  return(x+1)

[2, 4, 7, 48, 51]

In [ ]:
#aplica la funcion suma a cada uno de los elemento de column_movieID
column_movieId_suma = column_movieId.map(suma)
column_movieId_suma.take(5)

In [29]:
#Se pueden aplicar varias funciones a la vez y guardar en tuplas

column_movieId_variado = (column_movieId
                          .map(suma)
                          .map(lambda x: (x,x+2))
                          )

column_movieId_variado.take(5)

[(2, 4), (4, 6), (7, 9), (48, 50), (51, 53)]

In [30]:
#flatMap()
#transforma a una lista simple, lo aplana

column_movieId_flat = (column_movieId
                          .map(suma)
                          .flatMap(lambda x: (x,x+2))
                          )

column_movieId_flat.take(5)

[2, 4, 4, 6, 7]

# RDD clave - valor

## Desde archivo csv

In [36]:
# crear una lista de tuplas de un archivo

#abrir el archivo
df_2 = spark.read.csv("/content/datasets/movies.csv", header=True, inferSchema=True)

# Seleccionar la columna y convertirla a un RDD
lista_tupla =  df_2.rdd.map(lambda row: (row[1],row[0]))

lista_tupla.take(5)


[('Toy Story (1995)', 1),
 ('Jumanji (1995)', 2),
 ('Grumpier Old Men (1995)', 3),
 ('Waiting to Exhale (1995)', 4),
 ('Father of the Bride Part II (1995)', 5)]

## Desde archivo txt

In [41]:

lista_tupla1 = textfile.map(lambda x: (x.split(" ")[0],x))

lista_tupla1.collect()

[('primera', 'primera linea '),
 ('segunda', 'segunda linea'),
 ('tercera', 'tercera linea'),
 ('cuarta', 'cuarta linea')]

### zip

In [40]:
lista_tupla2 = sc.parallelize(zip(("a","b","c"),range(1,4,1)))

lista_tupla2.collect()

[('a', 1), ('b', 2), ('c', 3)]

## keyby

In [46]:
rdd = sc.parallelize(range(1,4,1))

lista_tupla3 = rdd.keyBy(lambda x: x+2 )
lista_tupla3.collect()

[(3, 1), (4, 2), (5, 3)]

## ZipWithIndex

In [49]:
#coloca id consecutivos comenzando de 0, pero no necesriamente unicos
rdd = sc.parallelize(["a","b","c"])

lista_tupla4 = rdd.zipWithIndex()
lista_tupla4.collect()

[('a', 0), ('b', 1), ('c', 2)]

## ZipWithUniqueID

In [58]:
# coloca id unicos y no necesariamente consecutivos

rdd = sc.parallelize(["a","b","c"])

lista_tupla5 = rdd.zipWithUniqueId()
lista_tupla5.collect()

[('a', 0), ('b', 1), ('c', 3)]